In [10]:
nx, ny = 100, 100
import numpy as np
C = np.empty([nx, ny], dtype='i')

In [12]:
C = np.empty([nx, ny], dtype='i').tobytes()
f = open("deneme.txt","w")
print(C,file = f)
f.close()

In [28]:
	xR = +2
	xL = -2
	yR = +2
	yL = -2
	nx = 500
	ny = 500
	Imax = 1000

In [12]:
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

In [25]:


amode = MPI.MODE_WRONLY|MPI.MODE_CREATE
fh = MPI.File.Open(comm, "./datafile.noncontig", amode)

item_count = 10

buffer = np.empty(item_count, dtype='i')
buffer[:] = rank

filetype = MPI.INT.Create_vector(item_count, 1, size)
filetype.Commit()

displacement = MPI.INT.Get_size()*rank
fh.Set_view(displacement, filetype=filetype)

fh.Write_all(buffer)
filetype.Free()
fh.Close()

In [26]:
#amode = MPI.MODE_RDONLY|MPI.MODE_CREATE
fh = MPI.File.Open(comm, "./datafile.noncontig", MPI.MODE_RDONLY)

In [15]:
fh.Read_all(buffer)

In [20]:
help(fh.Read)

Help on built-in function Read:

Read(...) method of mpi4py.MPI.File instance
    File.Read(self, buf, Status status=None)
    
    Read using individual file pointer



In [27]:
print(fh.Read([buffer,MPI.INT]))

None


In [29]:
fh.Get_position()

40

None


In [60]:
plane_coordinates = np.zeros(4, dtype='f')
pixel_maxiter = np.zeros(3, dtype='i')

if rank == 0:
    plane_coordinates[:] = [xL, xR, yL, yR]
    pixel_maxiter[:] = [nx, ny, Imax]
    
# Splitting data to other cores
comm.Bcast([plane_coordinates, MPI.FLOAT], root=0)
comm.Bcast([pixel_maxiter, MPI.INT], root=0)

xL, xR, yL, yR = [float(r) for r in plane_coordinates]
nx, ny, Imax    = [int(i) for i in pixel_maxiter]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [66]:
rank = np.array([0,1,2,3])
size = len(rank)
for i in rank:
    N = ny // size + (ny % size > i)
    N = np.array(N, dtype='i')
    I = np.arange(i, ny, size, dtype='i')
    print(I)

[  0   4   8  12  16  20  24  28  32  36  40  44  48  52  56  60  64  68
  72  76  80  84  88  92  96 100 104 108 112 116 120 124 128 132 136 140
 144 148 152 156 160 164 168 172 176 180 184 188 192 196 200 204 208 212
 216 220 224 228 232 236 240 244 248 252 256 260 264 268 272 276 280 284
 288 292 296 300 304 308 312 316 320 324 328 332 336 340 344 348 352 356
 360 364 368 372 376 380 384 388 392 396 400 404 408 412 416 420 424 428
 432 436 440 444 448 452 456 460 464 468 472 476 480 484 488 492 496]
[  1   5   9  13  17  21  25  29  33  37  41  45  49  53  57  61  65  69
  73  77  81  85  89  93  97 101 105 109 113 117 121 125 129 133 137 141
 145 149 153 157 161 165 169 173 177 181 185 189 193 197 201 205 209 213
 217 221 225 229 233 237 241 245 249 253 257 261 265 269 273 277 281 285
 289 293 297 301 305 309 313 317 321 325 329 333 337 341 345 349 353 357
 361 365 369 373 377 381 385 389 393 397 401 405 409 413 417 421 425 429
 433 437 441 445 449 453 457 461 465 469 473 477 481 4

In [33]:
help(np.arange)x

Help on built-in function arange in module numpy:

arange(...)
    arange([start,] stop[, step,], dtype=None)
    
    Return evenly spaced values within a given interval.
    
    Values are generated within the half-open interval ``[start, stop)``
    (in other words, the interval including `start` but excluding `stop`).
    For integer arguments the function is equivalent to the Python built-in
    `range` function, but returns an ndarray rather than a list.
    
    When using a non-integer step, such as 0.1, the results will often not
    be consistent.  It is better to use `numpy.linspace` for these cases.
    
    Parameters
    ----------
    start : number, optional
        Start of interval.  The interval includes this value.  The default
        start value is 0.
    stop : number
        End of interval.  The interval does not include this value, except
        in some cases where `step` is not an integer and floating point
        round-off affects the length of `out`.
   

In [67]:
C = np.empty([N, ny], dtype='i')
C, C.shape

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32), (125, 500))

In [68]:
dx = (xR - xL) / nx
dy = (yR - yL) / ny
dx, dy

(0.008, 0.008)

In [69]:
N, ny, nx

(array(125, dtype=int32), 500, 500)

In [70]:
%%time
print(5)

5
CPU times: user 160 µs, sys: 22 µs, total: 182 µs
Wall time: 143 µs


In [71]:
%time

for k in np.arange(N):
    y = yL + I[k] * dy
    for j in np.arange(ny):
        x = xL + j * dx
        C[k, j] = mandelbrot(x, y, Imax)
        print("rank", i, "-->", (k,j), "-->",(x,y))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs
rank 3 --> (0, 0) --> (-2.0, -1.976)
rank 3 --> (0, 1) --> (-1.992, -1.976)
rank 3 --> (0, 2) --> (-1.984, -1.976)
rank 3 --> (0, 3) --> (-1.976, -1.976)
rank 3 --> (0, 4) --> (-1.968, -1.976)
rank 3 --> (0, 5) --> (-1.96, -1.976)
rank 3 --> (0, 6) --> (-1.952, -1.976)
rank 3 --> (0, 7) --> (-1.944, -1.976)
rank 3 --> (0, 8) --> (-1.936, -1.976)
rank 3 --> (0, 9) --> (-1.928, -1.976)
rank 3 --> (0, 10) --> (-1.92, -1.976)
rank 3 --> (0, 11) --> (-1.912, -1.976)
rank 3 --> (0, 12) --> (-1.904, -1.976)
rank 3 --> (0, 13) --> (-1.896, -1.976)
rank 3 --> (0, 14) --> (-1.888, -1.976)
rank 3 --> (0, 15) --> (-1.88, -1.976)
rank 3 --> (0, 16) --> (-1.8719999999999999, -1.976)
rank 3 --> (0, 17) --> (-1.8639999999999999, -1.976)
rank 3 --> (0, 18) --> (-1.8559999999999999, -1.976)
rank 3 --> (0, 19) --> (-1.848, -1.976)
rank 3 --> (0, 20) --> (-1.84, -1.976)
rank 3 --> (0, 21) --> (-1.832, -1.976)
rank 3 --> (0, 22) --> (-1.824, -

rank 3 --> (1, 66) --> (-1.472, -1.944)
rank 3 --> (1, 67) --> (-1.464, -1.944)
rank 3 --> (1, 68) --> (-1.456, -1.944)
rank 3 --> (1, 69) --> (-1.448, -1.944)
rank 3 --> (1, 70) --> (-1.44, -1.944)
rank 3 --> (1, 71) --> (-1.432, -1.944)
rank 3 --> (1, 72) --> (-1.424, -1.944)
rank 3 --> (1, 73) --> (-1.416, -1.944)
rank 3 --> (1, 74) --> (-1.408, -1.944)
rank 3 --> (1, 75) --> (-1.4, -1.944)
rank 3 --> (1, 76) --> (-1.392, -1.944)
rank 3 --> (1, 77) --> (-1.384, -1.944)
rank 3 --> (1, 78) --> (-1.376, -1.944)
rank 3 --> (1, 79) --> (-1.3679999999999999, -1.944)
rank 3 --> (1, 80) --> (-1.3599999999999999, -1.944)
rank 3 --> (1, 81) --> (-1.3519999999999999, -1.944)
rank 3 --> (1, 82) --> (-1.3439999999999999, -1.944)
rank 3 --> (1, 83) --> (-1.3359999999999999, -1.944)
rank 3 --> (1, 84) --> (-1.3279999999999998, -1.944)
rank 3 --> (1, 85) --> (-1.3199999999999998, -1.944)
rank 3 --> (1, 86) --> (-1.3119999999999998, -1.944)
rank 3 --> (1, 87) --> (-1.3039999999999998, -1.944)
rank 3

rank 3 --> (2, 149) --> (-0.808, -1.912)
rank 3 --> (2, 150) --> (-0.8, -1.912)
rank 3 --> (2, 151) --> (-0.792, -1.912)
rank 3 --> (2, 152) --> (-0.784, -1.912)
rank 3 --> (2, 153) --> (-0.776, -1.912)
rank 3 --> (2, 154) --> (-0.768, -1.912)
rank 3 --> (2, 155) --> (-0.76, -1.912)
rank 3 --> (2, 156) --> (-0.752, -1.912)
rank 3 --> (2, 157) --> (-0.744, -1.912)
rank 3 --> (2, 158) --> (-0.736, -1.912)
rank 3 --> (2, 159) --> (-0.728, -1.912)
rank 3 --> (2, 160) --> (-0.72, -1.912)
rank 3 --> (2, 161) --> (-0.712, -1.912)
rank 3 --> (2, 162) --> (-0.704, -1.912)
rank 3 --> (2, 163) --> (-0.696, -1.912)
rank 3 --> (2, 164) --> (-0.688, -1.912)
rank 3 --> (2, 165) --> (-0.6799999999999999, -1.912)
rank 3 --> (2, 166) --> (-0.6719999999999999, -1.912)
rank 3 --> (2, 167) --> (-0.6639999999999999, -1.912)
rank 3 --> (2, 168) --> (-0.6559999999999999, -1.912)
rank 3 --> (2, 169) --> (-0.6479999999999999, -1.912)
rank 3 --> (2, 170) --> (-0.6399999999999999, -1.912)
rank 3 --> (2, 171) --> 

rank 3 --> (3, 233) --> (-0.1359999999999999, -1.88)
rank 3 --> (3, 234) --> (-0.1279999999999999, -1.88)
rank 3 --> (3, 235) --> (-0.11999999999999988, -1.88)
rank 3 --> (3, 236) --> (-0.11199999999999988, -1.88)
rank 3 --> (3, 237) --> (-0.10399999999999987, -1.88)
rank 3 --> (3, 238) --> (-0.09599999999999986, -1.88)
rank 3 --> (3, 239) --> (-0.08799999999999986, -1.88)
rank 3 --> (3, 240) --> (-0.08000000000000007, -1.88)
rank 3 --> (3, 241) --> (-0.07200000000000006, -1.88)
rank 3 --> (3, 242) --> (-0.06400000000000006, -1.88)
rank 3 --> (3, 243) --> (-0.05600000000000005, -1.88)
rank 3 --> (3, 244) --> (-0.04800000000000004, -1.88)
rank 3 --> (3, 245) --> (-0.040000000000000036, -1.88)
rank 3 --> (3, 246) --> (-0.03200000000000003, -1.88)
rank 3 --> (3, 247) --> (-0.02400000000000002, -1.88)
rank 3 --> (3, 248) --> (-0.016000000000000014, -1.88)
rank 3 --> (3, 249) --> (-0.008000000000000007, -1.88)
rank 3 --> (3, 250) --> (0.0, -1.88)
rank 3 --> (3, 251) --> (0.00800000000000000

rank 3 --> (4, 276) --> (0.20800000000000018, -1.848)
rank 3 --> (4, 277) --> (0.2160000000000002, -1.848)
rank 3 --> (4, 278) --> (0.2240000000000002, -1.848)
rank 3 --> (4, 279) --> (0.2320000000000002, -1.848)
rank 3 --> (4, 280) --> (0.2400000000000002, -1.848)
rank 3 --> (4, 281) --> (0.24800000000000022, -1.848)
rank 3 --> (4, 282) --> (0.2560000000000002, -1.848)
rank 3 --> (4, 283) --> (0.26400000000000023, -1.848)
rank 3 --> (4, 284) --> (0.27200000000000024, -1.848)
rank 3 --> (4, 285) --> (0.28000000000000025, -1.848)
rank 3 --> (4, 286) --> (0.28800000000000026, -1.848)
rank 3 --> (4, 287) --> (0.29600000000000026, -1.848)
rank 3 --> (4, 288) --> (0.30400000000000027, -1.848)
rank 3 --> (4, 289) --> (0.31199999999999983, -1.848)
rank 3 --> (4, 290) --> (0.31999999999999984, -1.848)
rank 3 --> (4, 291) --> (0.32799999999999985, -1.848)
rank 3 --> (4, 292) --> (0.33599999999999985, -1.848)
rank 3 --> (4, 293) --> (0.34399999999999986, -1.848)
rank 3 --> (4, 294) --> (0.351999

...

rank 3 --> (119, 253) --> (0.02400000000000002, 1.8320000000000003)
rank 3 --> (119, 254) --> (0.03200000000000003, 1.8320000000000003)
rank 3 --> (119, 255) --> (0.040000000000000036, 1.8320000000000003)
rank 3 --> (119, 256) --> (0.04800000000000004, 1.8320000000000003)
rank 3 --> (119, 257) --> (0.05600000000000005, 1.8320000000000003)
rank 3 --> (119, 258) --> (0.06400000000000006, 1.8320000000000003)
rank 3 --> (119, 259) --> (0.07200000000000006, 1.8320000000000003)
rank 3 --> (119, 260) --> (0.08000000000000007, 1.8320000000000003)
rank 3 --> (119, 261) --> (0.08800000000000008, 1.8320000000000003)
rank 3 --> (119, 262) --> (0.09600000000000009, 1.8320000000000003)
rank 3 --> (119, 263) --> (0.10400000000000009, 1.8320000000000003)
rank 3 --> (119, 264) --> (0.1120000000000001, 1.8320000000000003)
rank 3 --> (119, 265) --> (0.1200000000000001, 1.8320000000000003)
rank 3 --> (119, 266) --> (0.1280000000000001, 1.8320000000000003)
rank 3 --> (119, 267) --> (0.13600000000000012, 1.

rank 3 --> (120, 345) --> (0.7600000000000002, 1.8639999999999999)
rank 3 --> (120, 346) --> (0.7680000000000002, 1.8639999999999999)
rank 3 --> (120, 347) --> (0.7760000000000002, 1.8639999999999999)
rank 3 --> (120, 348) --> (0.7840000000000003, 1.8639999999999999)
rank 3 --> (120, 349) --> (0.7920000000000003, 1.8639999999999999)
rank 3 --> (120, 350) --> (0.8000000000000003, 1.8639999999999999)
rank 3 --> (120, 351) --> (0.8080000000000003, 1.8639999999999999)
rank 3 --> (120, 352) --> (0.8159999999999998, 1.8639999999999999)
rank 3 --> (120, 353) --> (0.8239999999999998, 1.8639999999999999)
rank 3 --> (120, 354) --> (0.8319999999999999, 1.8639999999999999)
rank 3 --> (120, 355) --> (0.8399999999999999, 1.8639999999999999)
rank 3 --> (120, 356) --> (0.8479999999999999, 1.8639999999999999)
rank 3 --> (120, 357) --> (0.8559999999999999, 1.8639999999999999)
rank 3 --> (120, 358) --> (0.8639999999999999, 1.8639999999999999)
rank 3 --> (120, 359) --> (0.8719999999999999, 1.8639999999999

rank 3 --> (121, 357) --> (0.8559999999999999, 1.896)
rank 3 --> (121, 358) --> (0.8639999999999999, 1.896)
rank 3 --> (121, 359) --> (0.8719999999999999, 1.896)
rank 3 --> (121, 360) --> (0.8799999999999999, 1.896)
rank 3 --> (121, 361) --> (0.8879999999999999, 1.896)
rank 3 --> (121, 362) --> (0.8959999999999999, 1.896)
rank 3 --> (121, 363) --> (0.9039999999999999, 1.896)
rank 3 --> (121, 364) --> (0.9119999999999999, 1.896)
rank 3 --> (121, 365) --> (0.9199999999999999, 1.896)
rank 3 --> (121, 366) --> (0.9279999999999999, 1.896)
rank 3 --> (121, 367) --> (0.9359999999999999, 1.896)
rank 3 --> (121, 368) --> (0.944, 1.896)
rank 3 --> (121, 369) --> (0.952, 1.896)
rank 3 --> (121, 370) --> (0.96, 1.896)
rank 3 --> (121, 371) --> (0.968, 1.896)
rank 3 --> (121, 372) --> (0.976, 1.896)
rank 3 --> (121, 373) --> (0.984, 1.896)
rank 3 --> (121, 374) --> (0.992, 1.896)
rank 3 --> (121, 375) --> (1.0, 1.896)
rank 3 --> (121, 376) --> (1.008, 1.896)
rank 3 --> (121, 377) --> (1.016, 1.896)

rank 3 --> (122, 353) --> (0.8239999999999998, 1.928)
rank 3 --> (122, 354) --> (0.8319999999999999, 1.928)
rank 3 --> (122, 355) --> (0.8399999999999999, 1.928)
rank 3 --> (122, 356) --> (0.8479999999999999, 1.928)
rank 3 --> (122, 357) --> (0.8559999999999999, 1.928)
rank 3 --> (122, 358) --> (0.8639999999999999, 1.928)
rank 3 --> (122, 359) --> (0.8719999999999999, 1.928)
rank 3 --> (122, 360) --> (0.8799999999999999, 1.928)
rank 3 --> (122, 361) --> (0.8879999999999999, 1.928)
rank 3 --> (122, 362) --> (0.8959999999999999, 1.928)
rank 3 --> (122, 363) --> (0.9039999999999999, 1.928)
rank 3 --> (122, 364) --> (0.9119999999999999, 1.928)
rank 3 --> (122, 365) --> (0.9199999999999999, 1.928)
rank 3 --> (122, 366) --> (0.9279999999999999, 1.928)
rank 3 --> (122, 367) --> (0.9359999999999999, 1.928)
rank 3 --> (122, 368) --> (0.944, 1.928)
rank 3 --> (122, 369) --> (0.952, 1.928)
rank 3 --> (122, 370) --> (0.96, 1.928)
rank 3 --> (122, 371) --> (0.968, 1.928)
rank 3 --> (122, 372) --> (

rank 3 --> (123, 438) --> (1.504, 1.96)
rank 3 --> (123, 439) --> (1.512, 1.96)
rank 3 --> (123, 440) --> (1.52, 1.96)
rank 3 --> (123, 441) --> (1.528, 1.96)
rank 3 --> (123, 442) --> (1.536, 1.96)
rank 3 --> (123, 443) --> (1.544, 1.96)
rank 3 --> (123, 444) --> (1.552, 1.96)
rank 3 --> (123, 445) --> (1.56, 1.96)
rank 3 --> (123, 446) --> (1.568, 1.96)
rank 3 --> (123, 447) --> (1.576, 1.96)
rank 3 --> (123, 448) --> (1.584, 1.96)
rank 3 --> (123, 449) --> (1.592, 1.96)
rank 3 --> (123, 450) --> (1.6, 1.96)
rank 3 --> (123, 451) --> (1.608, 1.96)
rank 3 --> (123, 452) --> (1.616, 1.96)
rank 3 --> (123, 453) --> (1.624, 1.96)
rank 3 --> (123, 454) --> (1.6320000000000001, 1.96)
rank 3 --> (123, 455) --> (1.6400000000000001, 1.96)
rank 3 --> (123, 456) --> (1.6480000000000001, 1.96)
rank 3 --> (123, 457) --> (1.6560000000000001, 1.96)
rank 3 --> (123, 458) --> (1.6640000000000001, 1.96)
rank 3 --> (123, 459) --> (1.6720000000000002, 1.96)
rank 3 --> (123, 460) --> (1.6800000000000002,

In [ ]:
N = ny // size + (ny % size > rank)
N = np.array(N, dtype='i')
# indices of lines to compute here
I = np.arange(rank, ny, size, dtype='i')

# compute local lines
C = np.empty([N, ny], dtype='i')
for k in np.arange(N):
    y = yL + I[k] * dy
    for j in np.arange(ny):
        x = xL + j * dx
        C[k, j] = mandelbrot(x, y, Imax)
        
# gather results at root
counts = 0
indices = None
cdata = None

if rank == 0:
	counts = np.empty(size, dtype='i')
	indices = np.empty(ny, dtype='i')
	cdata = np.empty([ny, nx], dtype='i')

comm.Gather(sendbuf=[N, MPI.INT],
            recvbuf=[counts, MPI.INT],
            root=0)

comm.Gatherv(sendbuf=[I, MPI.INT],
             recvbuf=[indices, (counts, None), MPI.INT],
             root=0)

comm.Gatherv(sendbuf=[C, MPI.INT],
             recvbuf=[cdata, (counts*nx, None), MPI.INT],
             root=0)


if rank == 0:
    M = np.zeros([ny,nx], dtype='i')
    M[indices, :] = cdata
    
    #plt.gray()
    #plt.imshow(M)
    #plt.show()
    
tend = MPI.Wtime()

wct = comm.gather(tend-tstart, root=0)
if rank == 0:
    for rank_id, time in enumerate(wct):
        print('wall clock time of processor %d is: %8.2f seconds' % (rank_id,time))


